In [2]:
# Exo 1 - Préparation des données 

# importation du fichier
import pandas as pd 
data = pd.read_csv("synthetic.csv")

# Visualisation des données
print(data.head())


Error: (1,3): error CS1024: Preprocessor directive expected
(3,3): error CS1024: Preprocessor directive expected
(4,15): error CS1002: ; expected
(4,15): error CS1525: Invalid expression term 'as'
(4,20): error CS1002: ; expected
(5,36): error CS1002: ; expected
(7,3): error CS1024: Preprocessor directive expected

In [ ]:
# Nombre de colonnes (attributs) dans le DataFrame
num_attributes = data.shape[1]

# Afficher le nombre d'attributs
print(f"Le nombre d'attributs dans le fichier est : {num_attributes}")


In [ ]:
# Type de données et valeurs manquantes
print(data.info())

In [ ]:
# Avoir le nombre d'attributs dans le modèle
print(data.columns)
# 14 attributs dans le modèle

In [ ]:
# Nombre de classes diff
nbr_class = data['Class'].nunique()
print(nbr_class)
# 4 classes différentes

In [ ]:
# combien d'instanes ompte haque classe?
nbr_instances = data['Class'].value_counts()
print(nbr_instances)

# Sortie 
# Class
# 1    908
# 0    674
# 2    472
# 3    244
# Name: count, dtype: int64

# Les données sont-elles linéairement séparables ?
Non, si on observe le schéma 1 on voit que les données ne le sont pas.
De plus si l'on choisit de les ranger par classe , on peut s'apercevoir que 


In [ ]:
import matplotlib.pyplot as plt # import biblio matplot
plt.figure(figsize=(10, 6))
plt.scatter(data['Attr_A'], data['Attr_B'], c=data['Class'], alpha=0.5, cmap='viridis')
plt.xlabel('Attribut 1')
plt.ylabel('Attribut 2')
plt.title('Scatter Plot des attributs par classe')
plt.colorbar(label='Classe')
plt.show()

# On peut voir clairement que ce n'est pas divisible linéairement à l'état brut
# je pense que use image est vraiment mieux


# 5 - 
- Pour l'arbre de décision
    Puisque les variables sont déjà numériques et que la colonne Class est utilisée comme étiquette (et non comme une fonctionnalité), aucun encodage One-hot n'est nécessaire pour les fonctionnalités. Si Class était utilisée comme une caractéristique d'entrée plutôt que comme une étiquette, et si elle comprenait de nombreuses catégories différentes, l'encodage One-hot pourrait être envisagé pour éviter de donner un ordre artificiel entre les catégories.
    Pour un modèle basé sur un arbre de décision, la normalisation des données n'est généralement pas nécessaire. Les arbres de décision ne sont pas sensibles à la magnitude des valeurs des attributs de la même manière que le sont les modèles basés sur des calculs de distance ou des modèles linéaires. Voici pourquoi :

- Pour le réseau de neurones :
    Dans notre jeu de données la colonne 'Class' est utilisé comme étiquette pour un problème de classification multi-classes.
    hot-one est recommandé afin d'éviter chaque classe comme une sortie distincte et de ne pas établir de relation d'ordre.
    Les réseaux de neurones étant sensibles aux échelles des variables d'entrée, car cela influence comment les poids sont ajustés durant l'apprentissage,normaliser les données est donc nécesssaire. Cela aide à accélérer la convergence lors de l'utilisation des algorithmes de descente de gradient.
    

    

# Partie 2 : Mise en oeuvre des modèles.